Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from timeit import default_timer as timer
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'


def load_data(data):
    global test_dataset
    test_dataset = data[0][1]
    global test_labels
    test_labels = data[0][2]
    global valid_dataset
    valid_dataset = data[1][1]
    global valid_labels
    valid_labels = data[1][2]
    global train_dataset
    train_dataset = data[2][1]
    global train_labels
    train_labels = data[2][2]
    
def print_data_shape():
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

with open(pickle_file, 'rb') as f:
    sets = pickle.load(f)
    load_data(sets['unsanitized'])
    del sets  # hint to help gc free up memory
    print_data_shape()
    


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

def make_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def make_xavier_weights(shape, name):
    return tf.get_variable(name, shape, initializer=tf.contrib.layers.xavier_initializer())

def make_bias(shape, constant=0.1):
    return tf.Variable(tf.constant(constant, shape=shape))

iteration_log = []
def log_step(start_time, duration, progress, loss, batch_acc, valid_acc=False, test_acc=False):
    iteration_log.append([start_time, duration, progress, loss, batch_acc, valid_acc, test_acc])
    print('>>>>>', int(progress * 100), '% done')
    print("time to execute last %d minibatches: %f" % (int(num_steps / num_prints), duration))
    print()
    print("loss of last minibatch %f" % loss)
    print("accuracy on last minibatch: %.1f%%" % batch_acc)
    print()
    if (valid_acc):
        print("model's current accuracy against validation data : %.1f%%" % valid_acc)
    if (test_acc):
        print("model's current accuracy against test data       : %.1f%%" % test_acc)
    print()

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
num_relus = 1024
beta_loss_coefficient = .0001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    
    step1_weights = make_weights([image_size * image_size, num_relus])
    step1_biases = make_bias([num_relus])
    
    step2_weights = make_weights([num_relus, num_labels])
    step2_biases = make_bias([num_labels])
    
    # Training computation.
    def run_on_data(data):
        step1_logits = tf.matmul(data, step1_weights) + step1_biases
        intermediate_step = tf.nn.relu(step1_logits)
        my_logits = tf.matmul(intermediate_step, step2_weights) + step2_biases
        return my_logits
    
    logits = run_on_data(tf_train_dataset)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
        + beta_loss_coefficient * tf.nn.l2_loss(step1_weights)
        + beta_loss_coefficient * tf.nn.l2_loss(step1_biases)
        + beta_loss_coefficient * tf.nn.l2_loss(step2_weights)
        + beta_loss_coefficient * tf.nn.l2_loss(step2_biases)
    )
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(run_on_data(tf_valid_dataset))
    test_prediction = tf.nn.softmax(run_on_data(tf_test_dataset))


In [6]:
num_steps = 500
num_prints = 5
beta_loss_coefficient = .0001

with tf.Session(graph=graph) as session:
    initial_time = timer()
    start = timer()
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized. # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch. # The key of the dictionary is the placeholder node of the graph to be fed, # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        # run
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % int(num_steps / num_prints) == 0):
            end = timer()
            log_step(start, end - start, step / num_steps, l,
              accuracy(predictions, batch_labels),
              accuracy(valid_prediction.eval(), valid_labels),
              accuracy(test_prediction.eval(), test_labels)
            )
            start = timer()
    print()
    print("Total time for %d iterations: %d" % (num_steps, timer() - initial_time))


Initialized
>>>>> 0 % done
time to execute last 100 minibatches: 1.818886

loss of last minibatch 4.274596
accuracy on last minibatch: 8.6%

model's current accuracy against validation data : 31.0%
model's current accuracy against test data       : 33.4%

>>>>> 20 % done
time to execute last 100 minibatches: 0.291903

loss of last minibatch 1.099306
accuracy on last minibatch: 78.9%

model's current accuracy against validation data : 81.7%
model's current accuracy against test data       : 88.9%

>>>>> 40 % done
time to execute last 100 minibatches: 0.255183

loss of last minibatch 1.090671
accuracy on last minibatch: 80.5%

model's current accuracy against validation data : 81.5%
model's current accuracy against test data       : 88.0%

>>>>> 60 % done
time to execute last 100 minibatches: 0.257726

loss of last minibatch 0.917587
accuracy on last minibatch: 82.8%

model's current accuracy against validation data : 83.9%
model's current accuracy against test data       : 90.3%

>>>>> 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 5
num_prints = 5
beta_loss_coefficient = .0001

with tf.Session(graph=graph) as session:
    initial_time = timer()
    start = timer()
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        
        # run
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        
        if (step % int(num_steps / num_prints) == 0):
            end = timer()
            log_step(start, end - start, step / num_steps, l,
              accuracy(predictions, batch_labels),
              accuracy(valid_prediction.eval(), valid_labels),
              accuracy(test_prediction.eval(), test_labels)
            )
            start = timer()
    print()
    print("Total time for %d iterations: %d" % (num_steps, timer() - initial_time))


Initialized
>>>>> 0 % done
time to execute last 1 minibatches: 0.251757

loss of last minibatch 4.704351
accuracy on last minibatch: 8.6%

model's current accuracy against validation data : 21.7%
model's current accuracy against test data       : 23.8%

>>>>> 20 % done
time to execute last 1 minibatches: 0.002989

loss of last minibatch 15.569920
accuracy on last minibatch: 25.8%

model's current accuracy against validation data : 40.5%
model's current accuracy against test data       : 43.9%

>>>>> 40 % done
time to execute last 1 minibatches: 0.003292

loss of last minibatch 11.649817
accuracy on last minibatch: 40.6%

model's current accuracy against validation data : 29.8%
model's current accuracy against test data       : 32.4%

>>>>> 60 % done
time to execute last 1 minibatches: 0.002858

loss of last minibatch 19.743181
accuracy on last minibatch: 34.4%

model's current accuracy against validation data : 41.4%
model's current accuracy against test data       : 45.1%

>>>>> 80 % 

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [8]:
batch_size = 128
num_relus = 1024
start_learning_rate = 0.1
beta_loss_coefficient = 0.001

layer_shape = [image_size * image_size, 1024, 1024, 1024, num_labels]

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    
    weights = [];
    biases = [];
    
    for i in range(len(layer_shape) - 1):
        weights.append(make_xavier_weights([layer_shape[i], layer_shape[i+1]], 'xavier%d'%i))
        biases.append(make_bias([layer_shape[i+1]]))
    
    # Training computation.
    def run_on_data(data, do_dropout = False):
        logits = tf.matmul(data, weights[0]) + biases[0]
        for i in range(len(layer_shape) - 2):
            hidden_layer = tf.nn.relu(logits)
            if (do_dropout):
                hidden_layer = tf.nn.dropout(hidden_layer, .5)
            logits = tf.matmul(hidden_layer, weights[i + 1]) + biases[i + 1]
        return logits
    
    logits = run_on_data(tf_train_dataset, True)
    
    unreduced_loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    for i in range(len(weights)):
        unreduced_loss = unreduced_loss + beta_loss_coefficient * tf.nn.l2_loss(weights[i])
        unreduced_loss = unreduced_loss + beta_loss_coefficient * tf.nn.l2_loss(biases[i])
    
    loss = tf.reduce_mean(unreduced_loss)
    
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(start_learning_rate, num_steps, 100000, 0.96, staircase=True)
    # Use simple momentum for the optimization.
#     optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=global_step)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(run_on_data(tf_valid_dataset))
    test_prediction = tf.nn.softmax(run_on_data(tf_test_dataset))


In [12]:
num_steps = 50000
num_prints = 50
beta_loss_coefficient = .001

with tf.Session(graph=graph) as session:
    initial_time = timer()
    start = timer()
    tf.global_variables_initializer().run()
    print("Initialized")
    accuracies = [[],[],[]] # train, valid, test
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        accuracies[0].append(accuracy(predictions, batch_labels))
#         accuracies[1].append(accuracy(valid_prediction.eval(), valid_labels))
#         accuracies[2].append(accuracy(test_prediction.eval(), test_labels))
        
        if (step % int(num_steps / num_prints) == 0):
            end = timer()
            accuracies[1].append(accuracy(valid_prediction.eval(), valid_labels))
            accuracies[2].append(accuracy(test_prediction.eval(), test_labels))
            log_step(start, end - start, step / num_steps, l, accuracies[0][-1], accuracies[1][-1], accuracies[2][-1])
            start = timer()
    print()
    print("Total time for %d iterations: %d" % (num_steps, timer() - initial_time))


Initialized
>>>>> 0 % done
time to execute last 1000 minibatches: 0.359141

loss of last minibatch 4.172360
accuracy on last minibatch: 8.6%

model's current accuracy against validation data : 23.2%
model's current accuracy against test data       : 24.4%

>>>>> 2 % done
time to execute last 1000 minibatches: 4.636552

loss of last minibatch 1.739068
accuracy on last minibatch: 82.8%

model's current accuracy against validation data : 84.8%
model's current accuracy against test data       : 91.3%

>>>>> 4 % done
time to execute last 1000 minibatches: 4.604278

loss of last minibatch 1.578790
accuracy on last minibatch: 81.2%

model's current accuracy against validation data : 85.9%
model's current accuracy against test data       : 92.3%

>>>>> 6 % done
time to execute last 1000 minibatches: 4.577176

loss of last minibatch 1.435704
accuracy on last minibatch: 84.4%

model's current accuracy against validation data : 86.9%
model's current accuracy against test data       : 92.9%

>>>>>